In [58]:
from openai import OpenAI
from dotenv import load_dotenv
from prompt import STORY_HIGHLIGHTS_GENERATOR, STORY_SUBPARTS_GENERATOR, STORY_SUBPARTS_DESCRIPTION_GENERATOR
import json
from pydantic import BaseModel
from typing import List
from pathvalidate import sanitize_filepath

load_dotenv()

client = OpenAI()

In [59]:
def addTokens(base, new):
    base['completion_tokens'] += new["completion_tokens"]
    base["prompt_tokens"] += new["prompt_tokens"]
    base["total_tokens"] += new["total_tokens"]
    return base

In [81]:

class Highlights(BaseModel):
    parts: list[str]
    summary: str

def highlights_generator(title, n):
    
    _input = f"""STORY TITLE: {title}\n\nN: {n}"""
    
    completion = client.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": STORY_HIGHLIGHTS_GENERATOR},
            {
                "role": "user",
                "content": _input
            }
        ],
        response_format=Highlights,
        temperature=1
    ).to_dict()

    out = completion["choices"][0]["message"]["content"]
    # print(out)
    highlights = json.loads(out)
    usage = completion["usage"]

    return highlights, usage

In [61]:
# highlights, usage = highlights_generator()

# print(highlights)
# print(usage)

In [82]:

class Subparts(BaseModel):
    subparts: list[str]
    summary: str

def subparts_generator(title, story_summary, assigned_part, sub_n):
    
    _input = f"""STORY TITLE: {title}\n\nSTORY_SUMMARY: {story_summary}\n\nASSIGNED_PART: {assigned_part}\n\nN: {sub_n}"""
    
    completion = client.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": STORY_SUBPARTS_GENERATOR},
            {
                "role": "user",
                "content": _input
            }
        ],
        response_format=Subparts,
        temperature=1
    ).to_dict()

    out = completion["choices"][0]["message"]["content"]
    # print(out)
    subparts = json.loads(out)
    usage = completion["usage"]

    return subparts, usage

In [ ]:
class SubpartsDescription(BaseModel):
    description: str

def subparts_description_generator(story_summary, part_summary, assigned_subpart):
    
    _input = f"""STORY_SUMMARY: {story_summary}\n\nPART_SUMMARY: {part_summary}\n\nASSIGNED_SUBPART: {assigned_subpart}"""
    
    completion = client.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": STORY_SUBPARTS_DESCRIPTION_GENERATOR},
            {
                "role": "user",
                "content": _input
            }
        ],
        response_format=SubpartsDescription,
        temperature=1
    ).to_dict()

    out = completion["choices"][0]["message"]["content"]
    # print(out)
    description = json.loads(out)
    usage = completion["usage"]

    return description, usage

In [83]:

def story_generator(title, n, sub_n):

    print(f"TITLE: {title}\nPART COUNT: {n}\nSUBPARTS_COUNT: {sub_n}\n\n")

    token_usage = {'completion_tokens': 0, 'prompt_tokens': 0, 'total_tokens': 0}
    
    story_path = sanitize_filepath(f"stories/{title.replace(' ', '_')}__{n}_{sub_n}.json")
    txt_path = story_path.replace(".json", ".txt")
    
    story_dict = {"title": title, "parts": {}}

    highlights, usage = highlights_generator(title=title, n=n)
    token_usage = addTokens(token_usage, usage)
    print(f"GENERATED HIGHLIGHTS OF THE STORY. TOKEN USAGE: {token_usage}")
    story_summary = highlights["summary"]
    story_dict["parts"] = {f"part{i+1}": {"part_title": part, "subparts": {}} for i, part in enumerate(highlights["parts"])}
    
    with open(story_path, "w") as outfile:
        json.dump(story_dict, outfile)

    for part, part_gist in story_dict["parts"].items():
        subparts, usage = subparts_generator(title=title, story_summary=story_summary, assigned_part=part_gist, sub_n=sub_n)
        token_usage = addTokens(token_usage, usage)
        part_summary = subparts["summary"]
        story_dict["parts"][part]["subparts"] = {f"subpart{i+1}": {"subpart_title": subpart, "description": ""} for i, subpart in enumerate(subparts["subparts"])}
        with open(story_path, "w") as outfile: 
            json.dump(story_dict, outfile)
        
        print(f"GENERATED SUBPARTS FOR {part}. TOKEN USAGE: {token_usage}")

        for subpart, subpart_gist in story_dict["parts"][part]["subparts"].items():
            description, usage = subparts_description_generator(story_summary=story_summary, part_summary=part_summary, assigned_subpart=subpart_gist)
            token_usage = addTokens(token_usage, usage)
            story_dict["parts"][part]["subparts"][subpart]["description"] = description["description"]
            with open(story_path, "w") as outfile:
                json.dump(story_dict, outfile)

            with open(txt_path, 'a') as f:
                f.write("\n" + description["description"])
            
            print(f"GENERATED SUBPART DESCRIPTION FOR {subpart}. TOKEN USAGE: {token_usage}")

    print(f"COMPLETED STORY. TOKEN USAGE: {token_usage}")
    story_dict["token_usage"] = token_usage
    with open(story_path, "w") as outfile:
        json.dump(story_dict, outfile)

    print(f"STORY SAVED AT {story_path}")
    
    return story_dict, token_usage


In [84]:
def multiple_stories_generator(titles, n, sub_n):

    # titles - list of string titles for stories, only alphanumeric names
    # n - number of parts for each story
    # sub_n - number of subparts for each part

    # NOTE: Each subpart adds almost 500 words in the story.

    for title in titles:
        assert len(title) > 0, ("Story titles must have at least one character")
        assert title.isalnum(), ("Story titles can only contain alphabetic or numeric values")
        assert len(title) <= 100, ("Story titles can only have less than 100 chars")
        
        story_generator(title, n, sub_n)


In [85]:
titles = ["F"]
n = 1
sub_n = 1

multiple_stories_generator(titles, n, sub_n)

TITLE: F
PART COUNT: 1
SUBPARTS_COUNT: 1


GENERATED HIGHLIGHTS OF THE STORY. TOKEN USAGE: {'completion_tokens': 156, 'prompt_tokens': 203, 'total_tokens': 359}
GENERATED SUBPARTS FOR part1. TOKEN USAGE: {'completion_tokens': 312, 'prompt_tokens': 613, 'total_tokens': 925}
GENERATED SUBPART DESCRIPTION FOR subpart1. TOKEN USAGE: {'completion_tokens': 1028, 'prompt_tokens': 1104, 'total_tokens': 2132}
COMPLETED STORY. TOKEN USAGE: {'completion_tokens': 1028, 'prompt_tokens': 1104, 'total_tokens': 2132}
STORY SAVED AT stories/F__1_1.json
